In [28]:
import itertools
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.cp_tensor import cp_to_tensor
import matplotlib.pyplot as plt

# Define your parameter ranges as lists
dt_list = [0.01]
nu_list = [500, 1000]
lambda_list = [50, 80, 100, 150, 180]
sigma_list = [4, 7, 10]
K_list = [100, 500, 1000]

# Generate the parameter grid
param_grid = list(itertools.product(dt_list, nu_list, lambda_list, sigma_list, K_list))

def alpha_sequence(alpha_0, alpha_K, K):
    ks = np.arange(K+1)
    alphas = alpha_0 + (ks / K) * (alpha_K - alpha_0)
    return alphas

def objective_function(particle, tensor, rank):
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))
    error = tl.norm(tensor - reconstructed_tensor)
    return error

def compute_consensus_point(particles, alpha, tensor, rank):
    min_energy_particle = min(particles, key=lambda p: objective_function(p, tensor, rank))
    min_energy = objective_function(min_energy_particle, tensor, rank)

    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        energy = objective_function(particle, tensor, rank)
        weight = np.exp(-alpha * (energy - min_energy))
        numerator_A += weight * particle['A']
        numerator_B += weight * particle['B']
        numerator_C += weight * particle['C']
        denominator += weight

    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator
    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}

def anisotropic_update(particles, consensus_point, lambda_, sigma, dt, tensor, rank):
    consensus_point_loss = objective_function(consensus_point, tensor, rank)
    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        current_loss = objective_function(particle, tensor, rank)
        if consensus_point_loss < current_loss:
            drift_A = (-lambda_) * (A - A_consensus) * dt
            drift_B = (-lambda_) * (B - B_consensus) * dt
            drift_C = (-lambda_) * (C - C_consensus) * dt
        else:
            drift_A = np.zeros_like(A)
            drift_B = np.zeros_like(B)
            drift_C = np.zeros_like(C)

        # Normal distribution noise
        B_A = np.random.normal(loc=0, scale=1, size=A.shape)
        B_B = np.random.normal(loc=0, scale=1, size=B.shape)
        B_C = np.random.normal(loc=0, scale=1, size=C.shape)

        diffusion_A = sigma * (A - A_consensus) * B_A * np.sqrt(dt)
        diffusion_B = sigma * (B - B_consensus) * B_B * np.sqrt(dt)
        diffusion_C = sigma * (C - C_consensus) * B_C * np.sqrt(dt)

        particle['A'] += drift_A + diffusion_A
        particle['B'] += drift_B + diffusion_B
        particle['C'] += drift_C + diffusion_C

    return particles

# Fixed parameters
I, J, K_tensor = 8, 8, 8
rank = 3
np.random.seed(42)
tensor = tl.tensor(np.random.random((I, J, K_tensor)))

alpha_0 = 1e3
alpha_final = 1e9

# Dictionary to store results for each parameter combination
results = []

for dt, nu, lambda_, sigma, K in param_grid:
    print(f"Running with dt={dt}, nu={nu}, lambda={lambda_}, sigma={sigma}, K={K}")

    # Initialize particles
    particles = []
    for _ in range(nu):
        A = np.random.randn(I, rank)
        B = np.random.randn(J, rank)
        C = np.random.randn(K_tensor, rank)
        particles.append({'A': A, 'B': B, 'C': C})

    # Generate alpha sequence for this K_val
    alpha_values = alpha_sequence(alpha_0, alpha_final, K)

    for iteration in range(K+1):
        alpha_current = alpha_values[iteration]
        consensus_point = compute_consensus_point(particles, alpha=alpha_current, tensor=tensor, rank=rank)
        consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
        consensus_error = tl.norm(tensor - consensus_tensor)

        # Update particles except for the last iteration
        if iteration < K:
            particles = anisotropic_update(particles, consensus_point, lambda_=lambda_, sigma=sigma, dt=dt, tensor=tensor, rank=rank)

    # Store the results
    results.append({
        'dt': dt,
        'nu': nu,
        'lambda': lambda_,
        'sigma': sigma,
        'K': K,
        'final_consensus_error': consensus_error
    })


Running with dt=0.01, nu=500, lambda=50, sigma=4, K=100
Running with dt=0.01, nu=500, lambda=50, sigma=4, K=500
Running with dt=0.01, nu=500, lambda=50, sigma=4, K=1000
Running with dt=0.01, nu=500, lambda=50, sigma=7, K=100
Running with dt=0.01, nu=500, lambda=50, sigma=7, K=500
Running with dt=0.01, nu=500, lambda=50, sigma=7, K=1000
Running with dt=0.01, nu=500, lambda=50, sigma=10, K=100
Running with dt=0.01, nu=500, lambda=50, sigma=10, K=500
Running with dt=0.01, nu=500, lambda=50, sigma=10, K=1000
Running with dt=0.01, nu=500, lambda=80, sigma=4, K=100
Running with dt=0.01, nu=500, lambda=80, sigma=4, K=500
Running with dt=0.01, nu=500, lambda=80, sigma=4, K=1000
Running with dt=0.01, nu=500, lambda=80, sigma=7, K=100
Running with dt=0.01, nu=500, lambda=80, sigma=7, K=500
Running with dt=0.01, nu=500, lambda=80, sigma=7, K=1000
Running with dt=0.01, nu=500, lambda=80, sigma=10, K=100
Running with dt=0.01, nu=500, lambda=80, sigma=10, K=500
Running with dt=0.01, nu=500, lambda=8

In [29]:
import pandas as pd
df_results = pd.DataFrame(results)
df_results

,dt,nu,lambda,sigma,K,final_consensus_error
0,0.01,500,50,4,100,11.149433
1,0.01,500,50,4,500,11.590173
2,0.01,500,50,4,1000,10.900646
3,0.01,500,50,7,100,7.726785
4,0.01,500,50,7,500,6.866807
...,...,...,...,...,...,...
85,0.01,1000,180,7,500,6.173629
86,0.01,1000,180,7,1000,6.055069
87,0.01,1000,180,10,100,13.288600
88,0.01,1000,180,10,500,6.441086


In [30]:
df_results.to_excel("8x8x8torank3.xlsx", index=False)